# Forecast Regional Demand & Supply variation | Genericart Medicine

### DATA INTEGRATION

Due To Company Policies, Nesessary Changes Ware Done Like Changing The Actual Medicine Names To False Names, 
Price Date & Outlet Location Ware Also Changed, to Preserve Data Privacy.
Below is Sample Dataset Containing Only 15% Of Actual Data

In [37]:
import pandas as pd
df = pd.read_csv("medicinal_sales_data_large.csv",  parse_dates=["date"],dayfirst=True)
df.head()

,date,product,location,quantity,price
0,2015-06-01,Medicine_4,Amravati,17,150.44
1,2015-06-01,Medicine_10,Mumbai,18,63.41
2,2015-06-01,Medicine_4,Jalna,14,150.44
3,2015-06-01,Medicine_4,Chandrapur,19,150.44
4,2015-06-01,Medicine_5,Mumbai,6,381.41


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 799129 entries, 0 to 799128
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   date      799129 non-null  object 
 1   product   799129 non-null  object 
 2   location  799129 non-null  object 
 3   quantity  799129 non-null  int64  
 4   price     799129 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 30.5+ MB


In [4]:
df.describe()

,quantity,price
count,799129.000000,799129.000000
mean,10.508850,230.266403
std,5.765023,135.895870
min,1.000000,61.250000
25%,6.000000,89.120000
50%,11.000000,173.760000
75%,16.000000,354.510000
max,20.000000,451.480000


In [5]:
df.shape

(799129, 5)

In [6]:
df.isnull().sum()

date        0
product     0
location    0
quantity    0
price       0
dtype: int64

In [14]:
df.columns

Index(['date', 'product', 'location', 'quantity', 'price'], dtype='object')

In [15]:
df.date.min() #Start Date 

'01-01-2016'

In [17]:
df.date.max() #End Date

'31-12-2017'

In [26]:
df['product'].nunique() #Number Of Medicines 

10

In [27]:
df['location'].nunique() #Number Of Locations 

20

## Libraries 

In [28]:
import numpy as np
from datetime import datetime
import random
np.random.seed(42)
random.seed(42)

In [33]:
df['date'] = pd.to_datetime(df['date']) # Change type To Date 

In [39]:
df['year_month'] = df['date'].dt.to_period('M') # Extract Month-Year

In [40]:
df.head()

,date,product,location,quantity,price,year_month
0,2015-06-01,Medicine_4,Amravati,17,150.44,2015-06
1,2015-06-01,Medicine_10,Mumbai,18,63.41,2015-06
2,2015-06-01,Medicine_4,Jalna,14,150.44,2015-06
3,2015-06-01,Medicine_4,Chandrapur,19,150.44,2015-06
4,2015-06-01,Medicine_5,Mumbai,6,381.41,2015-06


In [48]:
# Monthly Quantitative Data For Each Product & Location
monthly_data = df.groupby(['year_month', 'product', 'location'])['quantity'].sum().reset_index()
monthly_data['year_month'] = monthly_data['year_month'].astype(str)
monthly_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8600 entries, 0 to 8599
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   year_month  8600 non-null   object
 1   product     8600 non-null   object
 2   location    8600 non-null   object
 3   quantity    8600 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 268.9+ KB


In [94]:
monthly_data['date'] = pd.to_datetime(monthly_data['year_month'])
monthly_data.to_csv('monthly_data.csv')
monthly_data.head()

,year_month,product,location,quantity,date
0,2015-06,Medicine_1,Ahmednagar,959,2015-06-01
1,2015-06,Medicine_1,Akola,1060,2015-06-01
2,2015-06,Medicine_1,Amravati,1060,2015-06-01
3,2015-06,Medicine_1,Aurangabad,994,2015-06-01
4,2015-06,Medicine_1,Chandrapur,973,2015-06-01


In [51]:
# Choose a specific product and location
product_name = "Medicine_1"
location_name = "Mumbai"

In [52]:
# Filter the data
filtered = monthly_data[(monthly_data['product'] == product_name) & (monthly_data['location'] == location_name)].copy()
filtered.set_index('date', inplace=True)
filtered = filtered.sort_index()
filtered.head()

,year_month,product,location,quantity
date,,,,
2015-06-01,2015-06,Medicine_1,Mumbai,1093
2015-07-01,2015-07,Medicine_1,Mumbai,852
2015-08-01,2015-08,Medicine_1,Mumbai,964
2015-09-01,2015-09,Medicine_1,Mumbai,1032
2015-10-01,2015-10,Medicine_1,Mumbai,1058


In [55]:
# Target 
ts = filtered['quantity']
ts.head()

date
2015-06-01    1093
2015-07-01     852
2015-08-01     964
2015-09-01    1032
2015-10-01    1058
Name: quantity, dtype: int64

### ARIMA Model

In [57]:
from statsmodels.tsa.arima.model import ARIMA

arima_model = ARIMA(ts, order=(2, 1, 2)).fit()
arima_forecast = arima_model.forecast(steps=1)[0]
arima_forecast

C:\Users\ARBAJ\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\ARBAJ\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\ARBAJ\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\ARBAJ\AppData\Local\Temp\ipykernel_7964\3196010561.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  arima_forecast = arima_model.forecast(steps=1)[0]


1111.3934136364555

### Artificial Neural Network

In [58]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler

In [59]:
def create_lags(series, n_lags=3):
    df = pd.DataFrame({'y': series})
    for i in range(1, n_lags + 1):
        df[f'lag_{i}'] = df['y'].shift(i)
    return df.dropna()

In [60]:
lag_df = create_lags(ts, n_lags=3)

X = lag_df.drop("y", axis=1).values # Prepare the Dataset
y = lag_df["y"].values 

In [61]:
# Scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

In [62]:
# ANN
model_ann = Sequential()
model_ann.add(Dense(64, input_dim=X_scaled.shape[1], activation='relu'))
model_ann.add(Dense(32, activation='relu'))
model_ann.add(Dense(1))
model_ann.compile(loss='mse', optimizer='adam')
model_ann.fit(X_scaled, y_scaled, epochs=100, verbose=0)

In [64]:
model_ann.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                256       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2369 (9.25 KB)
Trainable params: 2369 (9.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [66]:
model_ann.save('ANN.h5') # Model Saving

C:\Users\ARBAJ\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [68]:
# Predict next
last_3 = ts[-3:].values.reshape(1, -1)
last_3_scaled = scaler_X.transform(last_3)
last_3_scaled

array([[ 0.6223092 ,  0.63013699, -0.59882583]])

In [69]:
ann_scaled_pred = model_ann.predict(last_3_scaled, verbose=0)
ann_scaled_pred

array([[0.26217183]], dtype=float32)

In [70]:
ann_forecast = scaler_y.inverse_transform(ann_scaled_pred)[0][0]
ann_forecast

659.1944

### LSTM Model

In [71]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [73]:
def prepare_lstm_data(series, n_steps):
    X, y = [], []
    for i in range(len(series) - n_steps):
        X.append(series[i:i+n_steps])
        y.append(series[i+n_steps])
    return np.array(X), np.array(y)

In [74]:
#Scaling
lstm_scaler = MinMaxScaler()
scaled_series = lstm_scaler.fit_transform(ts.values.reshape(-1, 1)).flatten()

In [75]:
n_steps = 3
X_lstm, y_lstm = prepare_lstm_data(scaled_series, n_steps)
X_lstm = X_lstm.reshape((X_lstm.shape[0], n_steps, 1))

In [76]:
model_lstm = Sequential()
model_lstm.add(LSTM(50, activation='relu', input_shape=(n_steps, 1)))
model_lstm.add(Dense(1))
model_lstm.compile(optimizer='adam', loss='mse')
model_lstm.fit(X_lstm, y_lstm, epochs=50, verbose=0)

In [78]:
last_seq = scaled_series[-n_steps:].reshape((1, n_steps, 1))
lstm_scaled_pred = model_lstm.predict(last_seq, verbose=0)[0][0]

In [79]:
lstm_forecast = lstm_scaler.inverse_transform([[lstm_scaled_pred]])[0][0]
lstm_forecast

935.9530211091043

### Ensemble with Decision Tree

In [80]:
from sklearn.tree import DecisionTreeRegressor

X_ensemble = np.array([[arima_forecast, ann_forecast, lstm_forecast]])
y_dummy = [ts[-1]]  # Placeholder actual

C:\Users\ARBAJ\AppData\Local\Temp\ipykernel_7964\3154273866.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_dummy = [ts[-1]]  # Placeholder actual


In [81]:
ensemble_model = DecisionTreeRegressor()
ensemble_model.fit(X_ensemble, y_dummy)

final_prediction = ensemble_model.predict(X_ensemble)[0]

### Output


In [82]:
print(f"Forecast for next month in {location_name} for {product_name}:")
print("ARIMA Forecast:", round(arima_forecast, 2))
print("ANN Forecast:", round(ann_forecast, 2))
print("LSTM Forecast:", round(lstm_forecast, 2))
print("Ensemble Final Forecast:", round(final_prediction, 2))

Forecast for next month in Mumbai for Medicine_1:
ARIMA Forecast: 1111.39
ANN Forecast: 659.19
LSTM Forecast: 935.95
Ensemble Final Forecast: 445.0


# Project Pipeline 

In [83]:
# Helper function to create lag features
def create_lags(series, n_lags=3):
    df = pd.DataFrame({'y': series})
    for i in range(1, n_lags + 1):
        df[f'lag_{i}'] = df['y'].shift(i)
    return df.dropna()

In [84]:
# Helper function for LSTM preparation
def prepare_lstm_data(series, n_steps):
    X, y = [], []
    for i in range(len(series) - n_steps):
        X.append(series[i:i+n_steps])
        y.append(series[i+n_steps])
    return np.array(X), np.array(y)

In [95]:
# All unique products
products = df['product'].unique()
products

array(['Medicine_4', 'Medicine_10', 'Medicine_5', 'Medicine_7',
       'Medicine_3', 'Medicine_2', 'Medicine_8', 'Medicine_1',
       'Medicine_6', 'Medicine_9'], dtype=object)

In [87]:
def predict_medicine_demand(location: str, year: int, month: int):
    predictions = []

    for product_name in products:
        try:
            # Filter by product and location
            ts_df = monthly_data[(monthly_data['product'] == product_name) &
                                 (monthly_data['location'] == location)].copy()
            
            
            ts_df.set_index('date', inplace=True)
            ts_df = ts_df.sort_index()
            ts = ts_df['quantity']

            # Skip if too little data
            if len(ts) < 6:
                continue

            # ----- ARIMA -----
            arima_model = ARIMA(ts, order=(2, 1, 2)).fit()
            arima_pred = arima_model.forecast(steps=1)[0]

            # ----- ANN -----
            lag_df = create_lags(ts, 3)
            X_ann = lag_df.drop("y", axis=1).values
            y_ann = lag_df["y"].values

            scaler_X = MinMaxScaler()
            scaler_y = MinMaxScaler()
            X_scaled = scaler_X.fit_transform(X_ann)
            y_scaled = scaler_y.fit_transform(y_ann.reshape(-1, 1))

            model_ann = Sequential()
            model_ann.add(Dense(64, input_dim=3, activation='relu'))
            model_ann.add(Dense(32, activation='relu'))
            model_ann.add(Dense(1))
            model_ann.compile(optimizer='adam', loss='mse')
            model_ann.fit(X_scaled, y_scaled, epochs=100, verbose=0)

            last_3 = ts[-3:].values.reshape(1, -1)
            ann_pred_scaled = model_ann.predict(scaler_X.transform(last_3), verbose=0)
            ann_pred = scaler_y.inverse_transform(ann_pred_scaled)[0][0]

            # ----- LSTM -----
            lstm_scaler = MinMaxScaler()
            series_scaled = lstm_scaler.fit_transform(ts.values.reshape(-1, 1)).flatten()
            X_lstm, y_lstm = prepare_lstm_data(series_scaled, 3)
            X_lstm = X_lstm.reshape((X_lstm.shape[0], 3, 1))

            model_lstm = Sequential()
            model_lstm.add(LSTM(50, activation='relu', input_shape=(3, 1)))
            model_lstm.add(Dense(1))
            model_lstm.compile(optimizer='adam', loss='mse')
            model_lstm.fit(X_lstm, y_lstm, epochs=50, verbose=0)

            last_seq = series_scaled[-3:].reshape((1, 3, 1))
            lstm_pred_scaled = model_lstm.predict(last_seq, verbose=0)[0][0]
            lstm_pred = lstm_scaler.inverse_transform([[lstm_pred_scaled]])[0][0]

            # ----- Ensemble -----
            X_ens = np.array([[arima_pred, ann_pred, lstm_pred]])
            y_dummy = [ts[-1]]  # dummy target for fit
            ens_model = DecisionTreeRegressor()
            ens_model.fit(X_ens, y_dummy)
            final_pred = ens_model.predict(X_ens)[0]

            predictions.append({
                "location": location,
                "product": product_name,
                "month": month,
                "year": year,
                "arima": round(arima_pred, 2),
                "ann": round(ann_pred, 2),
                "lstm": round(lstm_pred, 2),
                "ensemble_forecast": round(final_pred, 2)
            })

        except Exception as e:
            print(f"Error for {product_name} at {location}: {e}")

    return pd.DataFrame(predictions)


In [89]:
results = predict_medicine_demand("Pune", 2019, 1)
results

C:\Users\ARBAJ\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\ARBAJ\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\ARBAJ\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\ARBAJ\AppData\Local\Temp\ipykernel_7964\889401875.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  arima_pred = arima_model.forecast(steps=1)[0]
C:\Users\ARBAJ\AppData\Local\Temp\ip

C:\Users\ARBAJ\AppData\Local\Temp\ipykernel_7964\889401875.py:60: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_dummy = [ts[-1]]  # dummy target for fit
C:\Users\ARBAJ\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\ARBAJ\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\ARBAJ\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\ARBAJ\AppData\Local\Temp\ipyke

,location,product,month,year,arima,ann,lstm,ensemble_forecast
0,Pune,Medicine_4,1,2019,1057.19,1099.010010,899.56,355.0
1,Pune,Medicine_10,1,2019,913.43,908.830017,813.32,359.0
2,Pune,Medicine_5,1,2019,902.00,582.299988,924.89,480.0
3,Pune,Medicine_7,1,2019,704.42,1456.239990,797.43,351.0
4,Pune,Medicine_3,1,2019,996.61,721.369995,855.95,477.0
5,Pune,Medicine_2,1,2019,621.22,810.039978,885.23,310.0
6,Pune,Medicine_8,1,2019,1087.32,1170.469971,837.64,244.0
7,Pune,Medicine_1,1,2019,815.05,1072.560059,865.45,438.0
8,Pune,Medicine_6,1,2019,928.39,1379.810059,850.20,332.0
9,Pune,Medicine_9,1,2019,993.50,991.590027,937.63,485.0


## Lets Now Create Production Ready Code

In [90]:
#!pip install joblib keras scikit-learn pandas numpy statsmodels

     ------------------------------------ 301.8/301.8 KB 777.5 kB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 698.7 kB/s eta 0:00:00
     ---------------------------------------- 11.1/11.1 MB 1.0 MB/s eta 0:00:00
     ---------------------------------------- 9.8/9.8 MB 818.6 kB/s eta 0:00:00
     ------------------------------------ 243.2/243.2 KB 355.2 kB/s eta 0:00:00
     -------------------------------------- 209.7/209.7 KB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 66.5/66.5 KB 3.7 MB/s eta 0:00:00
     -------------------------------------- 297.3/297.3 KB 1.2 MB/s eta 0:00:00
     ------------------------------------ 135.6/135.6 KB 888.8 kB/s eta 0:00:00
     ---------------------------------------- 3.0/3.0 MB 818.3 kB/s eta 0:00:00
     ---------------------------------------- 41.2/41.2 MB 1.3 MB/s eta 0:00:00
     -------------------------------------- 232.9/232.9 KB 1.1 MB/s eta 0:00:00
     -----------------------------------

You should consider upgrading via the 'C:\Users\ARBAJ\.pyenv\pyenv-win\versions\3.10.5\python.exe -m pip install --upgrade pip' command.


### Libraries

In [108]:
import os
import pandas as pd
import numpy as np
import joblib
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from statsmodels.tsa.arima.model import ARIMA
import tensorflow as tf

In [110]:
# Prepare data
df = pd.read_csv("medicinal_sales_data_large.csv", dayfirst=True)
df['date'] = pd.to_datetime(df['date'], dayfirst=True)
df['year_month'] = df['date'].dt.to_period('M')
monthly_data = df.groupby(['year_month', 'product', 'location'])['quantity'].sum().reset_index()
monthly_data['year_month'] = monthly_data['year_month'].astype(str)
monthly_data['date'] = pd.to_datetime(monthly_data['year_month'])

In [111]:
products = df['product'].unique()
locations = df['location'].unique()

In [112]:
# Directories
os.makedirs("models", exist_ok=True)

In [ ]:
def create_lags(series, n_lags=3):
    df = pd.DataFrame({'y': series})
    for i in range(1, n_lags + 1):
        df[f'lag_{i}'] = df['y'].shift(i)
    return df.dropna()

In [114]:
def prepare_lstm_data(series, n_steps):
    X, y = [], []
    for i in range(len(series) - n_steps):
        X.append(series[i:i+n_steps])
        y.append(series[i+n_steps])
    return np.array(X), np.array(y)

In [115]:
# Training and saving
for product in products:
    for location in locations:
        try:
            ts_df = monthly_data[(monthly_data['product'] == product) & (monthly_data['location'] == location)].copy()
            ts_df.set_index('date', inplace=True)
            ts_df = ts_df.sort_index()
            ts = ts_df['quantity']

            if len(ts) < 6:
                continue

            # ARIMA
            arima_model = ARIMA(ts, order=(2, 1, 2)).fit()
            joblib.dump(arima_model, f"models/arima_{product}_{location}.pkl")

            # ANN
            lag_df = create_lags(ts)
            X_ann = lag_df.drop("y", axis=1).values
            y_ann = lag_df["y"].values

            scaler_X = MinMaxScaler()
            scaler_y = MinMaxScaler()
            X_scaled = scaler_X.fit_transform(X_ann)
            y_scaled = scaler_y.fit_transform(y_ann.reshape(-1, 1))

            ann_model = Sequential([
                Dense(64, input_dim=3, activation='relu'),
                Dense(32, activation='relu'),
                Dense(1)
            ])
            ann_model.compile(optimizer='adam', loss='mse')
            ann_model.fit(X_scaled, y_scaled, epochs=100, verbose=0)

            ann_model.save(f"models/ann_{product}_{location}.h5")
            joblib.dump(scaler_X, f"models/ann_scalerX_{product}_{location}.pkl")
            joblib.dump(scaler_y, f"models/ann_scalerY_{product}_{location}.pkl")

            # LSTM
            lstm_scaler = MinMaxScaler()
            scaled_ts = lstm_scaler.fit_transform(ts.values.reshape(-1, 1)).flatten()
            X_lstm, y_lstm = prepare_lstm_data(scaled_ts, 3)
            X_lstm = X_lstm.reshape((X_lstm.shape[0], 3, 1))

            lstm_model = Sequential([
                LSTM(50, activation='relu', input_shape=(3, 1)),
                Dense(1)])
            lstm_model.compile(optimizer='adam', loss='mse')
            lstm_model.fit(X_lstm, y_lstm, epochs=50, verbose=0)

            lstm_model.save(f"models/lstm_{product}_{location}.h5")
            joblib.dump(lstm_scaler, f"models/lstm_scaler_{product}_{location}.pkl")

        except Exception as e:
            print(f"Failed for {product} - {location}: {e}")


KeyboardInterrupt: 

In [ ]:
def predict_medicine_demand(location: str, year: int, month: int):
    results = []
    products=('Medicine_4', 'Medicine_10', 'Medicine_5', 'Medicine_7','Medicine_3', 'Medicine_2', 'Medicine_8', 'Medicine_1','Medicine_6', 'Medicine_9')
    for product in products:
        try:
            monthly_data=pd.read_csv('monthly_data.csv')
            # Get last 3 values from the data
            ts_df = monthly_data[(monthly_data['product'] == product) & (monthly_data['location'] == location)].copy()
            ts_df.set_index('date', inplace=True)
            ts_df = ts_df.sort_index()
            ts = ts_df['quantity']
            if len(ts) < 6:
                continue

            # Load ARIMA model
            arima_model = joblib.load(f"models/arima_{product}_{location}.pkl")
            arima_forecast = arima_model.forecast(steps=1)[0]

            # Load ANN
            ann_model = tf.keras.models.load_model(f"models/ann_{product}_{location}.h5")
            scaler_X = joblib.load(f"models/ann_scalerX_{product}_{location}.pkl")
            scaler_y = joblib.load(f"models/ann_scalerY_{product}_{location}.pkl")

            last_3 = ts[-3:].values.reshape(1, -1)
            last_scaled = scaler_X.transform(last_3)
            ann_pred_scaled = ann_model.predict(last_scaled, verbose=0)
            ann_forecast = scaler_y.inverse_transform(ann_pred_scaled)[0][0]

            # Load LSTM
            lstm_model = tf.keras.models.load_model(f"models/lstm_{product}_{location}.h5")
            lstm_scaler = joblib.load(f"models/lstm_scaler_{product}_{location}.pkl")

            series_scaled = lstm_scaler.transform(ts.values.reshape(-1, 1)).flatten()
            last_seq = series_scaled[-3:].reshape(1, 3, 1)
            lstm_pred_scaled = lstm_model.predict(last_seq, verbose=0)[0][0]
            lstm_forecast = lstm_scaler.inverse_transform([[lstm_pred_scaled]])[0][0]

            results.append({
                "Product": product,
                "ARIMA Pred": round(arima_forecast, 2),
                "ANN Pred": round(ann_forecast, 2),
                "LSTM Pred": round(lstm_forecast, 2),
                "Average Requirement":round((arima_forecast+ann_forecast+lstm_forecast)/3,2)
            })

        except Exception as e:
            print(f"Failed to predict for {product} - {location}: {e}")
    return pd.DataFrame(results)


## Testing

In [116]:
import warnings
warnings.filterwarnings('ignore')
forecast = predict_medicine_demand("Mumbai", 2016, 2)
forecast

,Product,ARIMA Pred,ANN Pred,LSTM Pred,Average Requirement
0,Medicine_4,1046.99,860.729980,882.23,929.98
1,Medicine_10,789.24,1147.160034,968.23,968.21
2,Medicine_5,954.38,849.650024,866.13,890.05
3,Medicine_7,1069.64,1069.780029,918.81,1019.41
4,Medicine_3,1330.69,626.719971,877.27,944.90
5,Medicine_2,1055.28,1101.150024,890.05,1015.49
6,Medicine_8,1007.04,1212.469971,873.57,1031.03
7,Medicine_1,1111.39,842.340027,921.11,958.28
8,Medicine_6,659.11,702.030029,853.15,738.10
9,Medicine_9,1058.63,699.140015,844.30,867.36


In [119]:
monthly_data[(monthly_data['year_month']=='2016-02')&(monthly_data['location']=='Mumbai')]

,year_month,product,location,quantity,date
1611,2016-02,Medicine_1,Mumbai,912,2016-02-01
1631,2016-02,Medicine_10,Mumbai,1023,2016-02-01
1651,2016-02,Medicine_2,Mumbai,921,2016-02-01
1671,2016-02,Medicine_3,Mumbai,1065,2016-02-01
1691,2016-02,Medicine_4,Mumbai,869,2016-02-01
1711,2016-02,Medicine_5,Mumbai,691,2016-02-01
1731,2016-02,Medicine_6,Mumbai,821,2016-02-01
1751,2016-02,Medicine_7,Mumbai,746,2016-02-01
1771,2016-02,Medicine_8,Mumbai,720,2016-02-01
1791,2016-02,Medicine_9,Mumbai,937,2016-02-01
